In [ ]:
import pandas as pd
import numpy as np 
import re, json, os , logging , random, subprocess
from lib.config import connstr
from lib.context import context
from lib.utils import * 
from lib.picklist_recommender import picklist_recommender
from lib.issue_provider import issue_provider
from lib.questionnaire_parser import questionnaire_parser
from lib.questionnaire_picklist_parser import questionnaire_picklist_parser
from lib.script_generator import script_generator   
import nltk
from nltk.corpus import stopwords  
from nltk.stem import PorterStemmer
ps = PorterStemmer()
sw=stopwords.words('english') 
ctx=context() 
ctx.logger.setLevel(logging.DEBUG)
config = {}
with open('config.json', 'r') as f: 
    config=json.loads(f.read())    
ctx.config=config 
import warnings
warnings.filterwarnings('ignore')
#df, code = generate_code_from_db(ctx, qgroup=4028)

In [ ]:
def norm(s):
    s=re.sub('\s','',s)
    return s 

df_template = pd.read_excel(r"C:\Users\timko\Downloads\BOD 23-01 Reporting Templates 5.7.25.xlsx", header=0, sheet_name='Sheet1',  usecols='A:B', dtype=str)  
headers = df_template[['Header']].to_dict('records') 

In [ ]:


d1 = df_template.to_dict('records')
d2 = df_template.to_dict('records')
for c in ['CDMPercentage', 'AppPercentage', 'ChallengesConstraints' ]:
    for r in d1:
        d2.append( {'Header': r['Header'], 'Field': r['Field']+c} )

d2
dff_template = pd.DataFrame(d2)
df_cdm = dff_template.loc[  dff_template['Field'].str.contains('CDM') ]
s=''

for i, r in dff_template.iterrows():
    field = r['Field']
    if 'Percentage' in field:
        s += f', @{field} INT NULL \n'
    else:
        s += f', @{field} NVARCHAR(4000) NULL \n'
 

In [ ]:
'\n,'.join( df_cdm['Field'].to_list() )
Fields = df_cdm['Field'].to_list()
s=''
for f in Fields:
    s += f', {f}=@{f}\n'
print(s)

In [ ]:
ag = set(sql_todf("  SELECT Component FROM [Component List] WHERE IsActive=1 AND Component LIKE '%Business Administration%' AND FK_PK_Component IS NULL  ", connstr)['Component']) 
ag = list(ag)
ag

In [44]:
cols = ['Agency'] + df_cdm['Field'].to_list()
lod = []
for i in range(1):
    d={}
    for c in cols:
        d[c]=i+2
        if 'Agency' in c:
            d[c]=ag[0]
    lod.append(d)

pd.DataFrame(lod).to_csv(r"C:\Users\timko\Downloads\BOD2301CDM.csv", index=False)

In [ ]:

s, ss = '',''
for i, r in dff.iterrows():
    field = r['Field']
    s += f', {field}'
    ss += f', @{field}'
print(s)
print(ss)

In [ ]:

df=sql_todf("  SELECT COLUMN_NAME, TABLE_NAME FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME IN ('fsma_FormMaster', 'fsma_DataCall', 'fsma_FormTypes', 'fsma_Reportingcycles') ", connstr)     
for i,r in df.iterrows():
    cn=r['COLUMN_NAME']
    tn=re.sub('[a-z]|_','',r['TABLE_NAME'])
    print(f"\t, {tn}.{cn}")

In [ ]:
df=sql_todf("   SELECT PK_Question, QGROUP, IdText, sortpos, 70420 as new FROM vwQuestions WHERE PK_FORM = '2025-Q1-CIO' ORDER BY QGROUP, sortpos ", connstr)     
qg = df.iloc[0]['QGROUP']
inc = 0
dic = {}
lod = []
for i,r in df.iterrows():
    inc=inc+1
    if qg != df.iloc[i]['QGROUP']:
        qg = df.iloc[i]['QGROUP']
        inc=inc+4 
    n=df.iloc[i]['new'] 
    dic[str(df.iloc[i]['PK_Question']) ] = str(df.iloc[i]['new'] + inc)
    lod.append({'pk':str(df.iloc[i]['PK_Question']) , 'new':str(df.iloc[i]['new'] + inc)})
dff=pd.DataFrame(lod)
dff.to_json(r'C:\Users\timko\source\repos\SledgeOmatic\SledgeOMatic\Tasks\cio-keys.json', orient='records')
dff



In [ ]:

with open(r'C:\Users\timko\source\repos\SledgeOmatic\SledgeOMatic\Tasks\keys.json', 'w', encoding='UTF-8') as f: 
    f.write( f'{str(json.dumps(dic))}' ) 

In [ ]:
http://localhost:57236/BOD2301CDM/2025/2025_A_BOD2301_1.aspx
http://localhost:57236/Maintenance/BOD_2301_CDMAdmin.aspx


In [ ]:
dfcdm=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOD2301_CDMDATA'  ", connstr)   
 
cols = dfcdm[2:]['COLUMN_NAME'].to_list()
for f in cols:
    col = """  ,{
        field: "1",
        title: "2"
    }"""
    col=col.replace('1',f)
    col=col.replace('2',to_proper(f).replace('Percentage', ' pct.'))
    print(col)
    # print(f', {f} AS [{to_proper(f)}]')


  ,{
        field: "Agency",
        title: "Agency"
    }
  ,{
        field: "POADiscoveredCDMPercentage",
        title: "POADiscovered CDM Pct."
    }
  ,{
        field: "POAVulnerabilityEnumCDMPercentage",
        title: "POAVulnerability Enum CDM Pct."
    }
  ,{
        field: "POACredScanCDMPercentage",
        title: "POACred Scan CDM Pct."
    }
  ,{
        field: "EndpointsCDMPercentage",
        title: "Endpoints CDM Pct."
    }
  ,{
        field: "NetworkingDevicesCDMPercentage",
        title: "Networking Devices CDM Pct."
    }
  ,{
        field: "MobileDevicesCDMPercentage",
        title: "Mobile Devices CDM Pct."
    }
  ,{
        field: "UnknownCDMPercentage",
        title: "Unknown CDM Pct."
    }
  ,{
        field: "OtherIODevicesCDMPercentage",
        title: "Other IODevices CDM Pct."
    }
  ,{
        field: "POVSignaturesUpdatedCDMPercentage",
        title: "POVSignatures Updated CDM Pct."
    }
  ,{
        field: "ImportGroup",
        title: "Impor

In [50]:
df=sql_todf("  SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME = 'BOD2301CDM_Agency'  ", connstr)   
df=normalize_if_df(df) 
df 
df_cdm = df.loc[ df['COLUMN_NAME'] .str.contains('CDM')]
df_cdm 
df

,COLUMN_NAME,DATA_TYPE,CHARACTER_MAXIMUM_LENGTH,HEADER,PK_PICKLISTTYPE,DT
0,PK_BOD2301CDM_Agency,INT,0,PK_BOD2301CDM_Agency,0,PK_BOD2301CDM_Agency INT
1,PK_OrgSubmission,INT,0,PK_Org Submission,0,PK_OrgSubmission INT
2,POADiscoveredCDMPercentage,INT,0,POADiscovered CDMPercentage,0,POADiscoveredCDMPercentage INT
3,POAVulnerabilityEnumCDMPercentage,INT,0,POAVulnerability Enum CDMPercentage,0,POAVulnerabilityEnumCDMPercentage INT
4,POACredScanCDMPercentage,INT,0,POACred Scan CDMPercentage,0,POACredScanCDMPercentage INT
5,EndpointsCDMPercentage,INT,0,Endpoints CDMPercentage,0,EndpointsCDMPercentage INT
6,NetworkingDevicesCDMPercentage,INT,0,Networking Devices CDMPercentage,0,NetworkingDevicesCDMPercentage INT
7,MobileDevicesCDMPercentage,INT,0,Mobile Devices CDMPercentage,0,MobileDevicesCDMPercentage INT
8,UnknownCDMPercentage,INT,0,Unknown CDMPercentage,0,UnknownCDMPercentage INT
9,OtherIODevicesCDMPercentage,INT,0,Other IODevices CDMPercentage,0,OtherIODevicesCDMPercentage INT


In [ ]:
df_fields = df[2:]
df_fields

In [ ]:
ntb = '<telerik:RadNumericTextBox ID="field" runat="server" MinValue="0"/>'
tb = '<asp:TextBox runat="server" ID="field" Value="" />'
lbl = '<asp:Label runat="server" ID="field" Text="" />'
frm , s = "<tr> <td>{A}</td> <td>{B}</td> <td>{C}</td> <td>{D}</td> <td>{E}</td> <td>{F}</td> <td>{G}</td> <td>{H}</td>  </tr>", ''
dist = 9 
for i in range(0,9):
    s = frm 
    
    s = s.replace('{D}', lbl.replace('field', df_fields.iloc[i]['COLUMN_NAME']))
    s = s.replace('{F}', ntb.replace('field', df_fields.iloc[i+9]['COLUMN_NAME'])) 
    s = s.replace('{H}', tb.replace('field', df_fields.iloc[i+9+9]['COLUMN_NAME'])) 
    s = s.replace('{A}', headers[i]['Header']  )
    print(s)

In [ ]:
for i, row in df_cdm.iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read()  
        DATA_TYPE=row['DATA_TYPE']
        COLUMN_NAME=row['COLUMN_NAME']
        HEADER=row['HEADER'] 
        s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
        
        #print(f"[{COLUMN_NAME}] [NVARCHAR](4000) NULL,")  
        #print('{ field: "'+COLUMN_NAME+'", title: "'+HEADER+'" },')
        #s=f'<CB:DataField DBColumnName="{COLUMN_NAME}"  ImportColumnName="{HEADER}" Require="false"  runat="server"/>'
        print (s)

In [ ]:
s, ss, u, uu = '','','',''
for i, r in df.iterrows():
    c = f'{normalize_var(r["COLUMN_NAME"])}'  
    field= r["COLUMN_NAME"]
    u += f", CASE WHEN {field} < 0 THEN 'DNS' ELSE {field} END AS {field}_DISPLAY -- DNS (DID NOT RESPOND)\n"
    if re.match('.*App.*|.*CDMP.*', c):
        s +=  f", {field} " # \n" 
        ss +=  f", {i}" # f", @{field} \n" 
    if 'DATE' in r["DATA_TYPE"]:
        uu += f", FORMAT({field}, 'MM/dd/yyyy') AS {field} \n" 
    else:
        uu += f', {field} \n'
print(s)
print(ss) 
 

In [ ]:
for i,r in df.iterrows():
    c = f'{normalize_var(r["COLUMN_NAME"])}' 
    dfld = f'<CB:DataField  DBColumnName="{c}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if 'Perc' in c:
        dfld = f'<CB:DataField  DBColumnName="{c}" Require="False" DataType="System.DateTime" NonDatePlaceholder="01/01/1900" ImportColumnName="{c}"  runat="server"/>  '
    # print(s) 
    print(dfld)

In [ ]:


for c in df.columns:
    n = f'{normalize_var(c)}' 
    s = f',{n} NVARCHAR(MAX) NULL ' 
    # print(s)
    dfld = f'<CB:DataField  DBColumnName="{n}" Require="False" ImportColumnName="{c}" runat="server"/>  '
    if c[-4:] == "Date":
        dfld = f'<CB:DataField  DBColumnName="{n}" Require="False" DataType="System.DateTime" NonDatePlaceholder="01/01/1900" ImportColumnName="{c}"  runat="server"/>  '
    print(dfld )

In [ ]:
df = pd.read_excel(r'C:\Users\Tim\Downloads\fedramp.xlsx', sheet_name='Agency - CSP Table', header=1, usecols='B:Z' )  
df.columns=[c.replace(' ' ,'') for c in df.columns] 
df
# df.to_excel(r'C:\Users\Tim\Downloads\fedramp_admin_import.xlsx', sheet_name='Admin Import', index=False)


In [ ]:
df = pd.read_excel(r'C:\Users\timko\Downloads\TLP RED - FY25 HVA List - CyberScope Upload.xlsx', sheet_name='Export', header=0, usecols='A:Z' )   
df['PK_HVA'] = df['HVA_ID'].apply(id)
df['TierPK'] = df['Tier'].apply(tier)
df

In [ ]:

up = "UPDATE fsma_HVAs SET Tier='t', AgencyRank='ar' WHERE PK_HVA = 'pk' ;"
sql=''
for i, row in df[:].iterrows():
    s=up.replace("'t'", row["TierPK"])
    s=s.replace("'pk'", row["PK_HVA"])
    s=s.replace("'ar'", str(row["Rank within FHE"]))
    
    sql=sql+s+'\n'
print( sql )


with open(r'y:\tkopp\tscript.sql', 'w', encoding='UTF-8') as f: 
    f.write( f'\n{sql}' ) 



In [ ]:
fields = pd.read_csv(f'{ctx.get_dest()}fedramp.csv') 
df=normalizedf(fields)
df

In [ ]:
ag = set(sql_todf("  SELECT Component FROM [Component List] WHERE IsActive=1 AND FK_PK_Component IS NULL  ", connstr)['Component']) 
ag = set(ag)
ag

In [ ]:
for i, row in df[:].iterrows():
    s=f",@{row['COLUMN_NAME']} {row['DATA_TYPE']}"
    print(s)

In [ ]:
def df_fromTemplateColumn(path): 
    lst=[]
    cols=[]
    with open(path, 'r') as f:  
        g=re.search("<Columns>(.*)<\/Columns", f.read() , flags=re.DOTALL)   
        lst=g.groups(0)[0].split('<CB:CBGridTemplateColumn')[1:] 
        for l in lst: 
            un = re.search('UniqueName="(\w+)"\s', l ).groups(0)[0] 
            ht = un
            if 'HeaderText' in l:
                try: ht = re.search('HeaderText="([A-Za-z\s]*)"', l ).groups(0)[0]  
                except:  pass 
            cols.append({  'UniqueName': un ,'HeaderText':ht   })
            
    return pd.DataFrame(cols)
df = df_fromTemplateColumn(r'C:\dev\CyberScope\CyberScopeBranch\CSwebdev\code\CyberScope\BODSCuBA\2024\2024_A_SCuBATenantInventory_1.aspx')
df

In [ ]:

for i,r in df.iterrows(): 
    un = r['UniqueName'] 
    ht = r['HeaderText'] 
    s = f'<CB:DataField  DBColumnName="{un.replace(" ","")}" Require="False" ImportColumnName="{ht}" runat="server"/>  '
    if "Date" in un: 
        s = f'<CB:DataField  DBColumnName="{un.replace(" ","")}" Require="False" ImportColumnName="{ht}"  runat="server"/>  '
    print(s)

In [ ]:
for i, r in df.iterrows(): 
    DATA_TYPE=r["DATA_TYPE"]
    COLUMN_NAME=r["COLUMN_NAME"]
    HEADER=r["HEADER"]
    PK_PICKLISTTYPE=str(r["PK_PICKLISTTYPE"] )
    s=f'<CB:DataField DBColumnName="{COLUMN_NAME}" ImportColumnName="{HEADER}" SprocParamName="{COLUMN_NAME}" runat="server"/>' 
    s=f',{COLUMN_NAME} '
    if PK_PICKLISTTYPE != '0': s=s.replace('runat',' PickListTypeID="'+ PK_PICKLISTTYPE +'" runat')   
    
    e=f',{COLUMN_NAME} AS [{HEADER}] '
    if PK_PICKLISTTYPE != '0': e=f', (SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={COLUMN_NAME}) AS [{HEADER}]' 
    if 'Date' in COLUMN_NAME:
        e=f" , CASE WHEN {COLUMN_NAME} LIKE '%2000%' THEN 'NA' ELSE FORMAT({COLUMN_NAME}, 'MM/dd/yyyy') END AS [{HEADER}]"
    print(s)

In [ ]:

for i, row in df[:].iterrows():
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        s=f.read()  
    DATA_TYPE=row['DATA_TYPE']
    COLUMN_NAME=row['COLUMN_NAME']
    HEADER=row['HEADER']
    PK_PICKLISTTYPE=row['PK_PICKLISTTYPE']
    s=s.replace('{COLUMN_NAME}', COLUMN_NAME)
    if 'INT' in DATA_TYPE:  
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME+'_Display')
        s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{COLUMN_NAME}" DataValueField="YN" runat="server" />')  
    else:
        s=s.replace('{Bind_COLUMN_NAME}', COLUMN_NAME )
        if 'Date' in COLUMN_NAME:  
            # print("const "+COLUMN_NAME+" = new CBDatePicker({ id: $(`[id$=_"+COLUMN_NAME+"]`).prop('id'), allowNa: true  }); ")  
            s=s.replace('<%-- --%>', f'<asp:HiddenField ID="{COLUMN_NAME}" runat="server" />') 
        else: 
            s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{COLUMN_NAME}" runat="server" />') 
    #print(f"[{COLUMN_NAME}] [NVARCHAR](4000) NULL,")  
    #print('{ field: "'+COLUMN_NAME+'", title: "'+HEADER+'" },')
    print (s)

In [ ]:
 
def get_gridcols(df,out=False):
    ldf=df.to_dict(orient='records') 
    code,ddl,dv,sel,dtf=[],[],[],[],[]
    with open(ctx.get_tempalte_dir()+'GridTemplateColumn.aspx', 'r') as f: 
        t=f.read() 
        
        for d in ldf: 
            col=d['COLUMN_NAME'] 
            s=t.replace("{COLUMN_NAME}",d['COLUMN_NAME'])  
            if 'INT' in d['DATA_TYPE'].upper():
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}_Display")
                s=s.replace('<%-- --%>', f'<telerik:RadDropDownList ID="{col}" DataValueField="YN" runat="server" />')
                ddl.append(f'BindDDL(e, "{col}", "{col}", "{col}")')
                sel.append(f',(SELECT DisplayValue FROM vwPICKLISTS WHERE PK_Picklist={col}) AS {col}') 
            else: 
                s=s.replace('{Bind_COLUMN_NAME}',f"{d['COLUMN_NAME']}")
                s=s.replace('<%-- --%>', f'<telerik:RadTextBox ID="{col}" runat="server" />')
                dv.append(f'DirectCast(e.Item.FindControl($"{col}"), RadTextBox).Text = drVal("{col}").ToString()')
                sel.append(f',{col}') 
            code.append(f'{s}')     
    code='\n'.join(code) 
    ddl='\n'.join(ddl)
    dv= '\n'.join(dv)
    sel= '\n'.join(sel)[1:] 
    return {'code':code, 'ddl':ddl,'dv':dv,'sel':sel, 'dtf': dtf}
  
 
p = get_gridcols(df, out=False)
p
print ( p['code'] ) 
 

In [ ]:
sql = get_gridcols(df)
print( sql['code'])

In [ ]:
def get_sql(df,out=False):
    parms=[]
    for i,r in df.iterrows():
        s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()}({r['CHARACTER_MAXIMUM_LENGTH']}) = NULL"
        if r['DATA_TYPE'].upper() == 'INT':
            s=f"@{r['COLUMN_NAME']} {r['DATA_TYPE'].upper()} = NULL"  
        parms.append(f',{s}')

    into=', '.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c])
    vals=', @'.join([c for c in df['COLUMN_NAME'] if 'PK_' not in c]) 
    ins=f'\n INSERT INTO @TABLE ({into}) \n VALUES (@{vals})'

    parms='\n'.join(parms)[:] 
    update=', '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])  
    ext=' AND ' + ' AND '.join([f'{c}=@{c}' for c in df['COLUMN_NAME'] if 'PK_' not in c ])

    sel=[] 
    exp=[]
    for i,r in df.iterrows():   
        s=f"\n, {r['COLUMN_NAME']} "
        if r['DATA_TYPE'].upper() == 'INT':   
            s=s+f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['COLUMN_NAME']}_Display]" 
            exp.append(f"\n, (SELECT DisplayValue FROM vwPicklists WHERE PK_Picklist={r['COLUMN_NAME']}) AS [{r['HEADER']}]")
        elif r['DATA_TYPE'].upper() == 'DATETIME':
            s=s+f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['COLUMN_NAME']}_Display] " 
            exp.append(f"\n, FORMAT({r['COLUMN_NAME']}, 'MM/dd/yyyy') AS [{r['HEADER']}]") 
        else:
            exp.append(f"\n, {r['COLUMN_NAME']} AS [{r['HEADER']}]")
        sel.append(''+s)
    sel=''.join(sel)  
    exp=''.join(exp)  
    return {'parms':parms,'ins':ins, 'update':update, 'sel':sel, 'ext':ext, 'exp':exp  }


# df = sql_todf("SELECT * FROM INFORMATION_SCHEMA.COLUMNS WHERE TABLE_NAME='FEDRAMP_UPLOAD'", ctx.connstr) 
# df = normalizedf(df) 
#  df.columns
d=get_sql(df, out=False)
print( d['ins'].replace(',','\n,')  )